In [2]:
wikipedia_page = 'List_of_postal_codes_of_Canada:_M'
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

%config IPCompleter.greedy = True
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

table_from_top = 1

trace = False
wikipedia_url = 'https://en.wikipedia.org/wiki/{}'.format(wikipedia_page)
page = requests.get(wikipedia_url)
soup = BeautifulSoup(page.content, 'lxml')
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[table_from_top - 1]

feature_names = []

header_row = table.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))
    feature_name.replace('\n', '')
    feature_names.append(feature_name)
    
def has_coords(tag):
    if tag.has_attr('class'):
        if tag['class'][0] == 'latitude' or tag['class'][0] == 'longitude':
            return True
    return False

def get_coords(child):
    coords = []
    for coord in child.find_all(has_coords):
        coords.append(coord.string)
    if coords:
        if trace:
            return 'C = {}'.format(' '.join(coords))
        else:
            return ' '.join(coords)
    else:
        return ''

samples = []
sample_rows = table.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.string
        if text:
            if trace:
                features.append('T = {}'.format(text))
            else:
                features.append(text)
            continue
        
        for child in feature_col.children:
            if child.name == 'span':
                if child.has_attr('class'):
                    if child['class'] == 'display:none':
                        continue
                if child.find_all(has_coords):
                    feature_value = get_coords(child)
                    if feature_value:
                        break
                    else:
                        continue
            if child.name == 'sup':
                continue
            if child.name == 'a':
                if child.string[0] == '[':
                    continue            
            if child.name == 'a':
                if trace:
                    feature_value = 'A = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            if child.name == 'font':
                if trace:
                    feature_value = 'F = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            try:
                content = child.contents
            except AttributeError:
                if child.isspace():
                    continue
                if trace:
                    feature_value = 'E = {}'.format(child)
                else:
                    feature_value = child
                break
        features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))

In [3]:
df = pd.DataFrame(samples)
df.columns = df.columns.str.replace('\n','')
df.columns = df.columns.str.replace('Postcode','PostalCode')
df = df.replace({'\n' : ''})
#df1 = df.T.reindex(['PostalCode','Borough','Neighbourhood']).T
df = df[df.Borough != 'Not assigned' ]
df = df.dropna()
df = df.reset_index(drop=True)
#df.head()

In [4]:
df = df.replace({'Not assigned': ''}, regex=True)
#df = df.groupby('PostalCode').agg({'Borough':'sum','Neighbourhood':','.join})
df = df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.astype(str))))

df = df.replace({'\n' : ','})
df['Neighbourhood'] = df['Neighbourhood'].replace(',', df['Borough'])
df.loc[df['Borough'] == "Queen's Park"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [5]:
df.shape

(103, 3)

In [6]:
df = df.replace({'\n,' : ''})
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood\n, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
import pandas as pd
dt = pd.read_csv('http://cocl.us/Geospatial_data')
dt.columns = dt.columns.str.replace('Postal Code','PostalCode')
dt.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
dfl = pd.merge(df, dt[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode', how='left')
dfl
#dfl.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood\n, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, East Birchmount Park\n, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West\n, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n",43.692657,-79.264848
